In [1]:
# Simply execute the file and the rosbag given in 'file_name' will be written in a npy file
# The target is 'write_to'
# You have to stop the main function manually

import rospy
import sensor_msgs.point_cloud2 as pc2

import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output
from subprocess import Popen
import sys
import subprocess, yaml

global frames_1
frames_1 = []
global frames_2
frames_2 = []
global number_messages

path = "/media/juli/98F29C83F29C6772/SemesterProject/data/training"
file_names = ["/1-dust", "/2-dust", "/3-dust", "/4-dust", "/5-dust", "/6-dust", "/7-dust",
              "/8-dust", "/9-smoke", "/10-smoke", "/11-smoke", "/12-smoke", "/13-smoke",
              "/14-smoke", "/15-smoke", "/16-smoke", "/17-smoke", "/18-smoke", "/19-smoke",
              "/20-smoke", "/21-smoke", "/22-smoke", "/23-smoke", "/24-smoke", "/25-smoke"]
bag_number = 24
path = "/home/juli/Downloads"
filenames = ["/kitti_2011_09_26_drive_0001_synced"]
bag_number = 0

In [2]:
# !!! Can only be ran once, then you need to RESTART the kernel

def callback_1(data):
    global frames_1
    frames_1.append(data)
    if len(frames_1) >= number_messages and len(frames_2) >= number_messages:
        rospy.signal_shutdown("Finished reading this bag.")
    
def callback_2(data):
    global frames_2
    frames_2.append(data)
    rospy.loginfo("Frame: %s of %s for /velodyne_points_dual. I heard %s points in this frame.", len(frames_2), number_messages, data.width)
    clear_output(wait=True)
    if len(frames_1) >= number_messages and len(frames_2) >= number_messages:
        rospy.signal_shutdown("Finished reading this bag.")
    
def listener():
    # In ROS, nodes are uniquely named. If two nodes with the same
    # node are launched, the previous one is kicked off. The
    # anonymous=True flag means that rospy will choose a unique
    # name for our 'listener' node so that multiple listeners can
    # run simultaneously.
    rospy.init_node('listener', anonymous=True)

    rospy.Subscriber("/velodyne_points", pc2.PointCloud2, callback_1, queue_size = 10)
    rospy.Subscriber("/velodyne_points_dual", pc2.PointCloud2, callback_2, queue_size = 10)

    # spin() simply keeps python from exiting until this node is stopped
    rospy.spin()

if __name__ == '__main__':
    pcore = Popen(['roscore'])
    global frames_1
    global frames_2
    global number_messages
    name = file_names[bag_number]
    if True:
        frames_1 = []
        frames_2 = []
        p = Popen(['rosbag','play',path + name + ".bag"]) # something long running
        info_dict = yaml.load(subprocess.Popen(['rosbag', 'info', '--yaml', path + name + ".bag"], stdout=subprocess.PIPE).communicate()[0])
        number_messages = info_dict['topics'][6]['messages']
        listener()
        print("Writing bag" + name + "...")
        frames_1 = [np.array([list(point) for point in pc2.read_points(frame, field_names = ("x", "y", "z", "intensity", "ring"))]) 
          for frame in frames_1]
        frames_2 = [np.array([list(point) for point in pc2.read_points(frame, field_names = ("x", "y", "z", "intensity", "ring"))]) 
          for frame in frames_2]
        print len(frames_1), len(frames_2)
        assert len(frames_1) == len(frames_2)
        write_frames = []
        for i, write_frame in enumerate(frames_1):
            #print(write_frame.shape)
            write_frame = np.concatenate((write_frame, frames_2[i]), axis=1)
            write_frames.append(write_frame)
        print(len(write_frames))
        print(write_frames[0].shape)
        p.terminate()
    pcore.terminate()
    np.save(path + name + ".npy", write_frames)
            

Writing bag/25-smoke...
781 781
781
(69504, 10)


In [3]:
old_frames = wframes
frames = old_frames
frames2 = [np.array([list(point) for point in pc2.read_points(frame, field_names = ("x", "y", "z", "intensity", "ring"))]) 
          for frame in old_frames]
print frames2[0].shape
np.save(write_to, frames2)
loaded = np.load(write_to)
print loaded[0].shape

(69504, 5)
(69504, 5)


In [ ]:
# Tryout without explicit listening
# Not working so far
import rosbag
bag = rosbag.Bag('/home/juli/data/QUT_data/rosbags/2.bag')
for topic, msg, t in bag.read_messages(topics=['/velodyne_points']):
    frames.append(msg)
bag.close()

In [7]:
a = np.zeros([69504,5])
b = np.ones([69504,5])
print(a.shape)
print(np.concatenate((a,b),axis=1).shape)


(69504, 5)
(69504, 10)
